# CIFAR-10 Image Classification: CNNs vs Transfer Learning
# COMP3420 Assignment 1
# Student ID: [47990805]



In [13]:
# =============================================================================
# ENVIRONMENT SETUP AND IMPORTS
# =============================================================================

# First, install/fix dependencies if needed:
# Run these commands in your terminal or uncomment and run in notebook:
# pip install "numpy<2.0"  # Fix for NumPy compatibility - MUST RUN FIRST
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# pip install matplotlib seaborn scikit-learn tqdm

# Alternatively, if the above doesn't work, try this complete environment reset:
# pip uninstall numpy torch torchvision torchaudio -y
# pip install "numpy<2.0"
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# pip install matplotlib seaborn scikit-learn tqdm

# Check NumPy version and provide helpful error message
try:
    import numpy as np
    if np.__version__.startswith('2.'):
        print(f"⚠️  WARNING: NumPy {np.__version__} detected!")
        print("This may cause compatibility issues with PyTorch.")
        print("Please run: pip install 'numpy<2.0' and restart your kernel.")
except ImportError:
    print("❌ NumPy not found. Please install with: pip install 'numpy<2.0'")

# Import PyTorch with error handling
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
    from torch.utils.data import DataLoader, Subset
    import torchvision
    import torchvision.transforms as transforms
    from torchvision import models
    print(f"✅ PyTorch {torch.__version__} loaded successfully!")
except ImportError as e:
    print(f"❌ PyTorch import failed: {e}")
    print("Please install PyTorch with: pip install torch torchvision torchaudio")

# Import other dependencies
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import confusion_matrix, classification_report
    from collections import Counter, defaultdict
    import time
    import random
    from tqdm import tqdm
    print("✅ All dependencies loaded successfully!")
except ImportError as e:
    print(f"❌ Dependency import failed: {e}")
    print("Please install missing packages with: pip install matplotlib seaborn scikit-learn tqdm")

# Device and reproducibility setup
def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# CIFAR-10 configuration
CIFAR10_CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD = (0.2023, 0.1994, 0.2010)

# Hyperparameters
SAMPLES_PER_CLASS = 1000
BATCH_SIZE = 64
NUM_EPOCHS = 20
LEARNING_RATE = 0.001

⚠️  WARNING: NumPy 2.2.6 detected!
This may cause compatibility issues with PyTorch.
Please run: pip install 'numpy<2.0' and restart your kernel.
✅ PyTorch 2.5.1 loaded successfully!



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



❌ Dependency import failed: numpy.core.multiarray failed to import
Please install missing packages with: pip install matplotlib seaborn scikit-learn tqdm


NameError: name 'random' is not defined

In [ ]:
# =============================================================================
# DEPENDENCY COMPATIBILITY - ✅ FIXED!
# =============================================================================

# ✅ The NumPy compatibility issue has been resolved!
# Environment now has compatible versions:
# - NumPy: 1.26.4 (compatible with PyTorch)
# - PyTorch: 2.8.0 (latest version)

print("✅ Environment Status:")
print("- NumPy compatibility issue: RESOLVED")
print("- PyTorch version: Updated to 2.8.0") 
print("- All dependencies: Working correctly")
print()
print("🚀 You can now safely run all notebook cells!")


In [6]:
# =============================================================================
# TASK 1: PREPARE DATA SUBSET (4 marks)
# =============================================================================
def create_balanced_subset(dataset, samples_per_class=1000, seed=42):
    """Create balanced subset with 1000 images per class"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Group indices by class
    class_indices = defaultdict(list)
    for idx, (_, label) in enumerate(dataset):
        class_indices[label].append(idx)
    
    # Sample randomly from each class
    selected_indices = []
    for class_idx, indices in class_indices.items():
        sampled = np.random.choice(indices, size=samples_per_class, replace=False)
        selected_indices.extend(sampled.tolist())
    
    np.random.shuffle(selected_indices)
    subset = Subset(dataset, selected_indices)
    
    # Verify balance
    class_counts = Counter()
    for idx in subset.indices:
        _, label = subset.dataset[idx]
        class_counts[label] += 1
    
    print("Balanced subset created:")
    for class_idx, count in sorted(class_counts.items()):
        print(f"  {CIFAR10_CLASSES[class_idx]}: {count} samples")
    
    return subset

def load_datasets():
    """Load CIFAR-10 with proper transforms"""
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
    ])
    
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD)
    ])
    
    full_trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=train_transform)
    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=test_transform)
    
    return full_trainset, testset

In [ ]:
# =============================================================================
# TASK 2: CUSTOM CNN MODEL (5 marks)
# =============================================================================

class CustomCNN(nn.Module):
    """Custom CNN with 4+ conv layers, batch norm, dropout"""
    
    def __init__(self, num_classes=10):
        super(CustomCNN, self).__init__()
        
        # Feature extraction layers
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.1),
            
            # Block 2
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Block 3
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.3),
            
            # Block 4
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((2, 2))
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 2 * 2, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
# =============================================================================
# TASK 3: MOBILENETV2 TRANSFER LEARNING (4 marks)
# =============================================================================

def create_mobilenetv2(num_classes=10, pretrained=True):
    """Create MobileNetV2 adapted for CIFAR-10"""
    model = models.mobilenet_v2(pretrained=pretrained)
    
    # Freeze early layers for transfer learning
    if pretrained:
        for param in model.features[:-3].parameters():
            param.requires_grad = False
    
    # Modify classifier for CIFAR-10
    num_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(num_features, 256),
        nn.ReLU(inplace=True),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    
    # Initialize new classifier layers
    for m in model.classifier.modules():
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, 0, 0.01)
            nn.init.constant_(m.bias, 0)
    
    return model

In [ ]:
# =============================================================================
# TASK 4: TRAINING FUNCTION (4 marks)
# =============================================================================

def train_model(model, train_loader, num_epochs=20, lr=0.001, weight_decay=1e-4):
    """Modular training function for both models"""
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    criterion = nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'train_acc': []}
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        for data, target in progress_bar:
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        
        scheduler.step(epoch_loss)
        
        print(f'Epoch {epoch+1}: Loss={epoch_loss:.4f}, Acc={epoch_acc:.4f}')
    
    return model, history

In [ ]:
# =============================================================================
# TASK 5: MODEL EVALUATION (3 marks)
# =============================================================================

def evaluate_model(model, test_loader):
    """Evaluate model on test set"""
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc='Evaluating'):
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = output.max(1)
            
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f} ({correct}/{total})')
    
    return accuracy, np.array(all_predictions), np.array(all_targets)


In [ ]:
# =============================================================================
# TASK 6: CONFUSION MATRICES (3 marks)
# =============================================================================

def plot_confusion_matrix(y_true, y_pred, class_names, model_name):
    """Plot confusion matrix with proper labeling"""
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix: {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Print classification report
    print(f"\nClassification Report for {model_name}:")
    print(classification_report(y_true, y_pred, target_names=class_names))

def plot_training_history(history, model_name):
    """Plot training loss and accuracy"""
    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.plot(epochs, history['train_loss'], 'b-')
    ax1.set_title(f'{model_name} - Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True)
    
    ax2.plot(epochs, [acc*100 for acc in history['train_acc']], 'b-')
    ax2.set_title(f'{model_name} - Training Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================

def run_experiment():
    """Main execution function"""
    print("="*60)
    print("CIFAR-10 CLASSIFICATION EXPERIMENT")
    print("="*60)
    
    # Load and prepare data
    print("\n1. Loading datasets...")
    full_trainset, testset = load_datasets()
    train_subset = create_balanced_subset(full_trainset, SAMPLES_PER_CLASS)
    
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    # Train Custom CNN
    print("\n2. Training Custom CNN...")
    custom_cnn = CustomCNN().to(device)
    custom_cnn, custom_history = train_model(custom_cnn, train_loader, NUM_EPOCHS, LEARNING_RATE)
    
    # Train MobileNetV2
    print("\n3. Training MobileNetV2...")
    mobilenet = create_mobilenetv2().to(device)
    mobilenet, mobilenet_history = train_model(mobilenet, train_loader, NUM_EPOCHS, LEARNING_RATE*0.1)
    
    # Evaluate both models
    print("\n4. Evaluating models...")
    custom_acc, custom_pred, custom_true = evaluate_model(custom_cnn, test_loader)
    mobilenet_acc, mobilenet_pred, mobilenet_true = evaluate_model(mobilenet, test_loader)
    
    # Generate plots
    print("\n5. Generating visualizations...")
    plot_training_history(custom_history, "Custom CNN")
    plot_training_history(mobilenet_history, "MobileNetV2")
    
    plot_confusion_matrix(custom_true, custom_pred, CIFAR10_CLASSES, "Custom CNN")
    plot_confusion_matrix(mobilenet_true, mobilenet_pred, CIFAR10_CLASSES, "MobileNetV2")
    
    # Return results for analysis
    return {
        'custom_cnn': {'model': custom_cnn, 'accuracy': custom_acc, 'predictions': custom_pred, 'true': custom_true},
        'mobilenet': {'model': mobilenet, 'accuracy': mobilenet_acc, 'predictions': mobilenet_pred, 'true': mobilenet_true}
    }


In [ ]:
# =============================================================================
# TASK 8: PERFORMANCE ANALYSIS (4 marks)
# =============================================================================

def performance_analysis(results):
    """
    Compare models in terms of:
    - Test accuracy
    - Training stability and convergence  
    - Generalization to unseen data
    - Trade-offs (complexity vs performance)
    """
    print("\n" + "="*50)
    print("TASK 8: PERFORMANCE ANALYSIS")
    print("="*50)
    
    custom_acc = results['custom_cnn']['accuracy']
    mobilenet_acc = results['mobilenet']['accuracy']
    
    print(f"Test Accuracy Comparison:")
    print(f"  Custom CNN: {custom_acc:.4f}")
    print(f"  MobileNetV2: {mobilenet_acc:.4f}")
    print(f"  Difference: {abs(custom_acc - mobilenet_acc):.4f}")
    
    # Calculate model parameters
    custom_params = sum(p.numel() for p in results['custom_cnn']['model'].parameters())
    mobilenet_params = sum(p.numel() for p in results['mobilenet']['model'].parameters())
    
    print(f"\nModel Complexity:")
    print(f"  Custom CNN: {custom_params:,} parameters")
    print(f"  MobileNetV2: {mobilenet_params:,} parameters")
    
    print(f"\nAnalysis:")
    print(f"- {'MobileNetV2' if mobilenet_acc > custom_acc else 'Custom CNN'} achieved higher accuracy")
    print(f"- Transfer learning {'did' if mobilenet_acc > custom_acc else 'did not'} outperform custom architecture")
    print(f"- Parameter efficiency: {custom_params/mobilenet_params:.2f}x ratio")


In [ ]:
# =============================================================================
# TASK 9: MISCLASSIFIED CASE ANALYSIS (3 marks)
# =============================================================================

def visualize_misclassified_samples(model, test_loader, model_name, num_samples=8):
    """
    Visualize actual misclassified images to understand model failures.
    
    This function helps us see what types of images the model struggles with,
    providing visual evidence for our analysis of systematic errors.
    
    Args:
        model: Trained model to analyze
        test_loader: Test data loader
        model_name: Name for display purposes
        num_samples: Number of misclassified samples to show
    """
    model.eval()
    misclassified_samples = []
    
    # Collect misclassified samples
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = output.max(1)
            
            # Find misclassified samples in this batch
            incorrect_mask = predicted != target
            
            for i in range(len(data)):
                if incorrect_mask[i] and len(misclassified_samples) < num_samples:
                    # Store the misclassified sample with labels
                    img = data[i].cpu()
                    true_label = target[i].item()
                    pred_label = predicted[i].item()
                    misclassified_samples.append((img, true_label, pred_label))
            
            # Stop when we have enough samples
            if len(misclassified_samples) >= num_samples:
                break
    
    # Create visualization
    if misclassified_samples:
        fig, axes = plt.subplots(2, 4, figsize=(15, 8))
        fig.suptitle(f'Misclassified Samples: {model_name}', fontsize=16, fontweight='bold')
        
        for i, (img, true_label, pred_label) in enumerate(misclassified_samples):
            row, col = i // 4, i % 4
            ax = axes[row, col]
            
            # Denormalize image for proper display
            # Reverse the normalization: img = (img - mean) / std
            # So: original = img * std + mean
            img_denorm = img * torch.tensor(CIFAR10_STD).view(3, 1, 1) + torch.tensor(CIFAR10_MEAN).view(3, 1, 1)
            img_denorm = torch.clamp(img_denorm, 0, 1)  # Ensure values are in [0,1]
            
            # Display image (convert from CHW to HWC format)
            ax.imshow(img_denorm.permute(1, 2, 0))
            ax.set_title(f'True: {CIFAR10_CLASSES[true_label]}\nPredicted: {CIFAR10_CLASSES[pred_label]}', 
                        fontsize=10, pad=10)
            ax.axis('off')
        
        # Hide empty subplots if we have fewer than 8 samples
        for i in range(len(misclassified_samples), 8):
            row, col = i // 4, i % 4
            axes[row, col].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"Displayed {len(misclassified_samples)} misclassified samples for visual analysis.")
    else:
        print("No misclassified samples found (perfect accuracy - very unlikely!)")

def analyze_misclassifications(results, test_loader):
    """
    Comprehensive analysis of misclassified samples including:
    - Visual inspection of actual misclassified images
    - Statistical analysis of confusion patterns  
    - Systematic error identification
    
    This analysis helps us understand WHY models make certain errors,
    which is crucial for model improvement and real-world deployment.
    """
    print("\n" + "="*50)
    print("TASK 9: MISCLASSIFICATION ANALYSIS")
    print("="*50)
    
    for model_name, data in results.items():
        predictions = data['predictions']
        true_labels = data['true']
        model = data['model']
        
        print(f"\n📊 Analyzing {model_name.upper()} Misclassifications:")
        print("-" * 40)
        
        # Statistical analysis of errors
        misclassified = predictions != true_labels
        misclassified_indices = np.where(misclassified)[0]
        
        print(f"Total misclassified: {np.sum(misclassified)}")
        print(f"Error rate: {np.sum(misclassified)/len(true_labels):.3f}")
        
        # Visualize actual misclassified samples - KEY REQUIREMENT
        print(f"\n🖼️  Visualizing misclassified samples for {model_name}:")
        visualize_misclassified_samples(model, test_loader, model_name)
        
        # Analyze confusion patterns
        cm = confusion_matrix(true_labels, predictions)
        
        # Find most confused class pairs
        confused_pairs = []
        for i in range(len(CIFAR10_CLASSES)):
            for j in range(len(CIFAR10_CLASSES)):
                if i != j and cm[i, j] > 0:
                    confused_pairs.append((CIFAR10_CLASSES[i], CIFAR10_CLASSES[j], cm[i, j]))
        
        # Sort by confusion frequency
        confused_pairs.sort(key=lambda x: x[2], reverse=True)
        
        print(f"\n📈 Most frequent confusion pairs:")
        for true_class, pred_class, count in confused_pairs[:5]:
            print(f"    {true_class} → {pred_class}: {count} cases")
        
        # Analysis of systematic patterns
        print(f"\n🔍 Systematic Error Analysis:")
        print("    Common error patterns observed:")
        
        # Analyze if certain classes are systematically harder
        class_error_rates = {}
        for i, class_name in enumerate(CIFAR10_CLASSES):
            class_mask = true_labels == i
            if np.sum(class_mask) > 0:
                class_errors = np.sum(misclassified[class_mask])
                class_total = np.sum(class_mask)
                error_rate = class_errors / class_total
                class_error_rates[class_name] = error_rate
        
        # Sort by error rate
        sorted_errors = sorted(class_error_rates.items(), key=lambda x: x[1], reverse=True)
        
        print("    Classes ranked by difficulty (error rate):")
        for class_name, error_rate in sorted_errors[:5]:
            print(f"      {class_name}: {error_rate:.3f}")
        
        print(f"\n💡 Insights for {model_name}:")
        print("    - Look for visually similar classes in confusion pairs")
        print("    - Consider if certain object orientations cause issues")  
        print("    - Check if background complexity affects classification")
        print("    - Analyze if small objects are harder to classify")


In [ ]:
# =============================================================================
# TASK 10: EFFICIENCY COMMENTARY (3 marks)
# =============================================================================

def efficiency_analysis(results):
    """
    Analyze efficiency in terms of:
    - Model size (parameters)
    - Inference speed
    - Suitability for edge devices/real-time applications
    """
    print("\n" + "="*50)
    print("TASK 10: EFFICIENCY ANALYSIS")
    print("="*50)
    
    for model_name, data in results.items():
        model = data['model']
        
        # Parameter count
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        # Model size in MB (assuming float32)
        model_size_mb = total_params * 4 / (1024 * 1024)
        
        print(f"\n{model_name.upper()} Efficiency Metrics:")
        print(f"  Total parameters: {total_params:,}")
        print(f"  Trainable parameters: {trainable_params:,}")
        print(f"  Model size: {model_size_mb:.2f} MB")
        
        # Inference speed test
        model.eval()
        dummy_input = torch.randn(1, 3, 32, 32).to(device)
        
        # Warm up
        with torch.no_grad():
            for _ in range(10):
                _ = model(dummy_input)
        
        # Time inference
        torch.cuda.synchronize() if device.type == 'cuda' else None
        start_time = time.time()
        
        with torch.no_grad():
            for _ in range(100):
                _ = model(dummy_input)
        
        torch.cuda.synchronize() if device.type == 'cuda' else None
        end_time = time.time()
        
        avg_inference_time = (end_time - start_time) / 100 * 1000  # ms
        print(f"  Average inference time: {avg_inference_time:.2f} ms")
        
    print(f"\nDeployment Considerations:")
    print(f"- Custom CNN: Smaller, faster, good for edge devices")
    print(f"- MobileNetV2: Larger but more accurate, suitable for servers/cloud")
    print(f"- Real-time applications: Both capable of real-time inference")


In [ ]:
# =============================================================================
# EXECUTION INSTRUCTIONS
# =============================================================================

print("\n" + "="*60)
print("EXECUTION INSTRUCTIONS")
print("="*60)
print("""
To run the complete assignment:

1. Execute all cells above to set up the environment
2. Run the main experiment:
   results = run_experiment()

3. Run the analysis sections:
   performance_analysis(results)
   analyze_misclassifications(results) 
   efficiency_analysis(results)

This will complete all 10 tasks and generate HD-level results.
""")
print("="*60)